In [ ]:
#!pip install -U pip setuptools wheel
#!pip install -U spacy
#!python -m spacy download es_core_news_md
#!pip install Unidecode
#!pip install -U transformers
#!pip install torch
#!pip install sentencepiece
#!pip install summa
#!python -m pip install wordcloud
#!pip install nlpaug
#!pip install python-Levenshtein

In [ ]:
import spacy
nlp = spacy.load('es_core_news_md')
from spacy.lang.es.stop_words import STOP_WORDS
from spacy.matcher import Matcher
from spacy.tokens import Doc, Span
from spacy.matcher import PhraseMatcher
import string
from spacy.lang.es import Spanish
from spacy.language import Language
from heapq import nlargest
import re
import unidecode
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from summa.summarizer import summarize
import numpy as np
from wordcloud import WordCloud
import nltk
nltk.download('omw')
import nlpaug.augmenter.word as naw
aumento = naw.SynonymAug(aug_src='wordnet', lang='spa')
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC

from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix 
from sklearn import metrics
from sklearn.metrics import classification_report

from time import time

In [ ]:
def limpieza(documento): 
    """Función para limpiar textos. Solo requiere que se ingrese el texto (en formato string)
    La salida de la función es una lista con los tokens limpios"""
    texto = unidecode.unidecode(documento)
    texto = re.sub("\d+", ' ', texto)
    texto = re.sub("\\s+", ' ', texto)
    regex = '[\\!\\"\\#\\$\\%\\&\\\'\\(\\)\\*\\+\\,\\-\\.\\/\\:\\;\\<\\=\\>\\?\\@\\[\\\\\\]\\^_\\`\\{\\|\\}\\~]'
    texto = re.sub(regex, "", texto)
    documento = nlp(texto)
    tokens_limpios = [token.lemma_ for token in documento]
    tokens_limpios = [token for token in tokens_limpios if nlp(token)[0].is_punct == False]
    tokens_limpios = [token for token in tokens_limpios if nlp(token)[0].is_stop == False]
    tokens_minusculas = list(map(str.lower, tokens_limpios))

    return tokens_minusculas

In [ ]:
def grafico_top(documento, tipo=1):
    
    """
    Función que genera un gráfico de las palabras más frecuentes en un texto.
    Si la variable tipo es igual a 1 generará un gráfico de barras.
    Si la variable tipo es igual a 2 generará una nube de palabras.
    Por defecto genera un gráfico de barras.
    """
    
    tokens = limpieza(documento)
    frecuencia = [tokens.count(token) for token in tokens]
    lista = dict(zip(tokens, frecuencia))
    llaves_ordenadas = sorted(lista, key=lista.get)
    palabras = []
    valores = []

    for i in reversed(llaves_ordenadas):
        valor = lista.get(i)
        largo = len(i)
        if valor > 1:
            if largo > 2:
                if nlp(i)[0].pos_ != 'PROPN':
                    palabras.append(i)
                    valores.append(lista.get(i))

    if tipo == 1:
        fig, ax = plt.subplots(figsize=(24 ,3))
        sns.barplot(x=palabras[0:11], y=valores[0:11])
        plt.ylabel('Cantidad')
        plt.xlabel('Palabras')
        plt.ylim(0, max(valores)+1)
        plt.title('Palabras más usadas')
        plt.show()
    elif tipo == 2:
        wordcloud=WordCloud(collocations = False, background_color='white').generate(" ".join(tokens))
        plt.imshow(wordcloud, interpolation='bilinear')
        plt.axis("off")
        plt.show()

In [ ]:
def resumen(parrafo, porcentaje = 0.15):
    sintesis = summarize(parrafo, language='spanish', ratio = porcentaje).replace("\n", " ")
    return sintesis

## Descriptivo

In [ ]:
muestra1 = pd.read_excel('entrevistas_juicio.xlsx') 
extension_casos=[]
for a in range(0, len(muestra1.Juicio)):
    total = [token.text.replace('\n', ' ').lower() for token in nlp(muestra1.Juicio[a]).sents]
    extension_casos.append(len(total))

In [ ]:
print(sum(extension_casos))
print(min(extension_casos))
print(max(extension_casos))

In [ ]:
nom_casos=['caso1', 'caso2', 'caso3', 'caso4', 'caso5', 'caso6', 'caso7', 'caso8', 'caso9', 'caso10']
fig, ax = plt.subplots(figsize=(24 ,3))
sns.barplot(x=nom_casos, y=extension_casos, palette='gray')
plt.ylabel('Extensión (frases)')
plt.xlabel('Casos')
plt.ylim(0, max(extension_casos))
plt.title('Extensión de casos usados en el juicio de expertos')
plt.show()

## Validez de contenido

### Extracción frases de jueces

In [ ]:
juicio = pd.read_excel('Juicio_expertos.xlsx')
juez1 = juicio.juez1
juez2 = juicio.juez2
juez3 = juicio.juez3

In [ ]:
Datos_juicio= juez1
total_frases=[]
num_de_caso=[]
for n in range(0, len(Datos_juicio)):
    frasedj = [token.text.replace('\n', ' ').lower() for token in nlp(Datos_juicio[n]).sents]
    for w in frasedj:
        total_frases.append(w)
        num_de_caso.append(n)
df_juez_1=pd.DataFrame({'caso': num_de_caso, 'frases':total_frases})
df_juez_1.to_excel('resultados/frasesjuez1.xlsx')

In [ ]:
Datos_juicio= juez2
total_frases=[]
num_de_caso=[]
for n in range(0, len(Datos_juicio)):
    frasedj = [token.text.replace('\n', '  ').lower() for token in nlp(Datos_juicio[n]).sents]
    for w in frasedj:
        total_frases.append(w)
        num_de_caso.append(n)
df_juez_1=pd.DataFrame({'caso': num_de_caso, 'frases':total_frases})
df_juez_1.to_excel('resultados/frasesjuez2.xlsx')

In [ ]:
Datos_juicio= juez3
total_frases=[]
num_de_caso=[]
for n in range(0, len(Datos_juicio)):
    frasedj = [token.text.replace('\n', ' ').lower() for token in nlp(Datos_juicio[n]).sents]
    for w in frasedj:
        total_frases.append(w)
        num_de_caso.append(n)
df_juez_1=pd.DataFrame({'caso': num_de_caso, 'frases':total_frases})
df_juez_1.to_excel('resultados/frasesjuez3.xlsx')

#### Matriz de comparación entre jueces

In [ ]:
def comparacion(lista1, lista2, porc=0.75):
    '''
    Función de prueba
    '''
    frase_similar=[]
    caso_frase=[]
    porc_sim=[]
    matriz = np.empty((0, len(lista2)))
    for m in range(len(lista1)):
        for n in range(len(lista2)):
            porcentaje=nlp(" ".join(limpieza(lista1[m]))).similarity(nlp(" ".join(limpieza(lista2[n]))))
            porc_sim.append(round(porcentaje, 2))
            if porcentaje >= porc:
                frase_similar.append(lista1[m])
                caso_frase.append(m)
        matriz=np.append(matriz, np.array([porc_sim]), axis=0)
        porc_sim=[]
    return matriz, caso_frase, frase_similar 

In [ ]:
def similitud(a , b, porc=0.75):
  """
  La función genera una matríz de similitud de frases contenidas en dos párrafos. Estos deben estar contenidos en DataFrames.
  Requiere 3 parámetros:
  a = primer párrafo
  b = segundo párrafo
  porc = porcentaje de similitud mínima aceptada. Por defecto extrae el 75%.
  La función arroja tres resultados:
  1. lista_df: Una lista de matrices con los resultados de las comparaciones (valores entre 0 y 1).
  2. lista_frases: Una lista de las frases que tienen un porcentaje de similitud igual o mayor al establecido en porc.
  3. lista_caso: Una lista de los casos que contienen las frases obtenidas en la lista_frases.
  """
    lista_df=[]
    lista_frases=[]
    lista_caso=[]
    for n in range(0, len(a)):
    caso1j1= a[n]
    caso1j2= b[n]
    frasec1 = [token.text.replace('\n', ' ').lower() for token in nlp(caso1j1).sents]
    frasec2 = [token.text.replace('\n', ' ').lower() for token in nlp(caso1j2).sents] 
    matriz = np.empty((0, len(frasec2)))
    lista_porc = []
    for i in range(0, len(frasec1)):
        for j in range(0, len(frasec2)):
        porcentaje=nlp(" ".join(limpieza(frasec1[i]))).similarity(nlp(" ".join(limpieza(frasec2[j]))))
        if porcentaje >= porc:
            if frasec1[i] not in lista_frases:
                lista_frases.append(frasec1[i])
                lista_caso.append(n)
        lista_porc.append(round(porcentaje, 2))
        matriz=np.append(matriz, np.array([lista_porc]), axis=0)
        lista_porc = []
    resultados = pd.DataFrame(matriz)
    lista_df.append(resultados)

    return lista_df, lista_frases, lista_caso

In [ ]:
listaresultados, listafrases, listacasos = similitud(juez1,juez2)

In [ ]:
listaresultados2, listafrases2, listacasos2 = similitud(juez1,juez3)

In [ ]:
listaresultados3, listafrases3, listacasos3 = similitud(juez2,juez3)

In [ ]:
juez1y2 = pd.ExcelWriter('resultados/juez1y2.xlsx')
largo=len(listaresultados)
for i in range(0, largo):
    listaresultados[i].to_excel(juez1y2, sheet_name=str(i))
juez1y2.save()
juez1y2.close()

In [ ]:
juez1y3 = pd.ExcelWriter('resultados/juez1y3.xlsx')
largo=len(listaresultados2)
for i in range(0, largo):
    listaresultados2[i].to_excel(juez1y3, sheet_name=str(i))
juez1y3.save()
juez1y3.close()

In [ ]:
juez2y3 = pd.ExcelWriter('resultados/juez2y3.xlsx')
largo=len(listaresultados3)
for i in range(0, largo):
    listaresultados3[i].to_excel(juez2y3, sheet_name=str(i))
juez2y3.save()
juez2y3.close()

In [ ]:
TotalFrases = listafrases+listafrases2+listafrases3
TotalCasos = listacasos+listacasos2+listacasos3

In [ ]:
concordancia = pd.DataFrame({'caso': TotalCasos, 'frases':TotalFrases})

In [ ]:
concordancia.drop_duplicates(inplace=True)

In [ ]:
concordancia.to_excel('resultados/concordancia.xlsx')  

In [ ]:
Datos_juicio= muestra1.Juicio
total_frases=[]
num_de_caso=[]
for n in range(0, len(Datos_juicio)):
    frasedj = [token.text.replace('\n', ' ').lower() for token in nlp(Datos_juicio[n]).sents]
    for w in frasedj:
        total_frases.append(w)
        num_de_caso.append(n)
frases_juicio_original=pd.DataFrame({'caso': num_de_caso, 'frases':total_frases})
frases_juicio_original.to_excel('resultados/frases_juicio_original.xlsx')

# Etiquetado de casos

In [ ]:
concordancia = pd.read_excel('resultados/concordancia.xlsx')
JuicioOriginal = pd.read_excel('resultados/frases_juicio_original.xlsx')

In [ ]:
len(concordancia)

In [ ]:
num_caso=[]
frases=[]
relevancia=[]
for caso in range(0,10):
    base1=JuicioOriginal[JuicioOriginal['caso']== caso]
    base2=concordancia[concordancia['caso'] == caso]
    for frase in base1['frases']:
        for juicio in base2['frases']:
            porcentaje=nlp(" ".join(limpieza(frase))).similarity(nlp(" ".join(limpieza(juicio))))
            if porcentaje >= 0.75:
                relevancia.append(1)
                break
        num_caso.append(caso)
        frases.append(frase)
        if len(frases) != len(relevancia):
            relevancia.append(0)

In [ ]:
clasificacion = pd.DataFrame({'caso':num_caso,'frases':frases,'relevancia':relevancia})
clasificacion.to_excel('resultados/frasesclasificadas1.xlsx', index=False)

In [ ]:
clasificacion = pd.read_excel('resultados/frasesclasificadas1.xlsx')
clasificacion.groupby('relevancia').size()

### Aumentando los datos

In [ ]:
caso_aum=[]
frase_aum=[]
relevancia_aum=[]
for caso, frase, relevancia in zip(list(clasificacion['caso']), list(clasificacion['frases']), 
                                   list(clasificacion['relevancia'])):
    texto_aumentado= aumento.augment(frase, n=6) #libreria nlpaug
    for a in texto_aumentado:
        caso_aum.append(caso)
        frase_aum.append(a)
        relevancia_aum.append(relevancia)
clasificacion_aum = pd.DataFrame({'caso':caso_aum, 'frases': frase_aum, 'relevancia': relevancia_aum})

In [ ]:
clasificacion_final = pd.concat([clasificacion, clasificacion_aum], axis=0).reset_index(drop=True)

In [ ]:
clasificacion_final.to_excel('resultados/clasificacion_final.xlsx', index=False)

In [ ]:
clasificacion_final = pd.read_excel('resultados/clasificacion_final.xlsx')

In [ ]:
clasificacion_final.groupby('relevancia').size()

# Entrenando el modelo

## CountVectorizer

In [ ]:
display(clasificacion_final.shape)
round(clasificacion_final['relevancia'].value_counts()/clasificacion_final.shape[0],3)*100

In [ ]:
clasificacion_final['texto_limpio'] = clasificacion_final['frases'].apply(limpieza)

In [ ]:
clasificacion_final['texto_limpio'] = clasificacion_final['texto_limpio'].apply(" ".join)

In [ ]:
clasificacion_final.to_excel('resultados/clasificacion_final_ext.xlsx', index=False)

In [ ]:
clasificacion_final = pd.read_excel('resultados/clasificacion_final_ext.xlsx')

In [ ]:
clasificacion_final.dropna(inplace=True)

In [ ]:
X=clasificacion_final.texto_limpio
y=clasificacion_final.relevancia
print(X.shape, y.shape)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

In [ ]:
vect = CountVectorizer()
X_train_dtm = vect.fit_transform(X_train)
X_test_dtm = vect.transform(X_test)
print(X_train_dtm.shape, X_test_dtm.shape)

### Naive Bayes

In [ ]:
nb = MultinomialNB()
%time nb.fit(X_train_dtm, y_train)
y_pred_class = nb.predict(X_test_dtm)

In [ ]:
nb.get_params()

In [ ]:
confusion = confusion_matrix(y_test, y_pred_class)
print(confusion)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
cax = ax.matshow(confusion)
plt.title('Matriz de confusión modelo Naive Bayes')
plt.annotate(confusion[0,0], (0,0))
plt.annotate(confusion[0,1], (1,0), color = 'white')
plt.annotate(confusion[1,0], (0,1), color = 'white')
plt.annotate(confusion[1,1], (1,1))
fig.colorbar(cax)
ax.set_xticklabels([''] + ['Irrelevante', 'Relevante'])
ax.set_yticklabels([''] + ['Irrelevante', 'Relevante'])
plt.xlabel('Valores predichos')
plt.ylabel('Valores actuales')
plt.show()

In [ ]:
reporte = classification_report(y_test, y_pred_class, target_names=['Irrelevante','Relevante'], output_dict=True)
df_reporte=pd.DataFrame(reporte).transpose()
df_reporte.to_excel('reporte_clasificacion_NB.xlsx')
df_reporte

### SVC

In [ ]:
classifier = LinearSVC(class_weight='balanced')
%time classifier.fit(X_train_dtm, y_train)
y_pred_class_svc = classifier.predict(X_test_dtm)

In [ ]:
classifier.get_params()

In [ ]:
confusion_svc = confusion_matrix(y_test, y_pred_class_svc)
print(confusion_svc)
fig = plt.figure()
ax = fig.add_subplot(111)
cax = ax.matshow(confusion_svc)
plt.title('Matriz de confusión modelo SVM')
plt.annotate(confusion_svc[0,0], (0,0))
plt.annotate(confusion_svc[0,1], (1,0), color = 'white')
plt.annotate(confusion_svc[1,0], (0,1), color = 'white')
plt.annotate(confusion_svc[1,1], (1,1))
fig.colorbar(cax)
ax.set_xticklabels([''] + ['Irrelevante', 'Relevante'])
ax.set_yticklabels([''] + ['Irrelevante', 'Relevante'])
plt.xlabel('Valores predichos')
plt.ylabel('Valores actuales')
plt.show()

In [ ]:
print('\nClassification Report\n')
reporte=classification_report(y_test, y_pred_class_svc, target_names=['Irrelevante','Relevante'], output_dict=True)
df_reporte=pd.DataFrame(reporte).transpose()
df_reporte.to_excel('reporte_clasificacion_SVM.xlsx')
df_reporte

#### Implementando el modelo

In [ ]:
PruebaModelo = pd.read_excel('resultados/prueba_prediccion.xlsx')

In [ ]:
frases=PruebaModelo.oraciones
datos=vect.transform(frases)
prediccionnb=nb.predict(datos)
prediccionsvc=classifier.predict(datos)

### NB

In [ ]:
confusionpruebaNB = confusion_matrix(PruebaModelo.Relevancia, prediccionnb)
fig = plt.figure()
ax = fig.add_subplot(111)
cax = ax.matshow(confusionpruebaNB)
plt.title('Matriz de confusión prueba Naive Bayes')
plt.annotate(confusionpruebaNB[0,0], (0,0), color = 'white')
plt.annotate(confusionpruebaNB[0,1], (1,0), color = 'white')
plt.annotate(confusionpruebaNB[1,0], (0,1))
plt.annotate(confusionpruebaNB[1,1], (1,1))
fig.colorbar(cax)
ax.set_xticklabels([''] + ['Irrelevante', 'Relevante'])
ax.set_yticklabels([''] + ['Irrelevante', 'Relevante'])
plt.xlabel('Valores predichos')
plt.ylabel('Valores actuales')
plt.show()

In [ ]:
reportepruebaNB = classification_report(PruebaModelo.Relevancia, prediccionnb, target_names=['Irrelevante','Relevante'], output_dict=True)
df_reporte_NB=pd.DataFrame(reportepruebaNB).transpose()
df_reporte_NB.to_excel('reporte_prueba_NB.xlsx')
df_reporte_NB

### SVM

In [ ]:
confusionpruebaSVM = confusion_matrix(PruebaModelo.Relevancia, prediccionsvc)
fig = plt.figure()
ax = fig.add_subplot(111)
cax = ax.matshow(confusionpruebaSVM)
plt.title('Matriz de confusión prueba SVM')
plt.annotate(confusionpruebaSVM[0,0], (0,0), color = 'white')
plt.annotate(confusionpruebaSVM[0,1], (1,0), color = 'white')
plt.annotate(confusionpruebaSVM[1,0], (0,1))
plt.annotate(confusionpruebaSVM[1,1], (1,1), color = 'white')
fig.colorbar(cax)
ax.set_xticklabels([''] + ['Irrelevante', 'Relevante'])
ax.set_yticklabels([''] + ['Irrelevante', 'Relevante'])
plt.xlabel('Valores predichos')
plt.ylabel('Valores actuales')
plt.show()

In [ ]:
reportepruebaSVM = classification_report(PruebaModelo.Relevancia, prediccionsvc, target_names=['Irrelevante','Relevante'], output_dict=True)
df_reporte_SVM=pd.DataFrame(reportepruebaSVM).transpose()
df_reporte_SVM.to_excel('reporte_prueba_SVM.xlsx')
df_reporte_SVM

In [ ]:
PruebaModelo['prediccionNB'] = list(prediccionnb)
PruebaModelo['prediccionSVC'] = list(prediccionsvc)
PruebaModelo['validacionNB']  = [1 if PruebaModelo.Relevancia[i] == PruebaModelo.prediccionNB[i] else 0 for i in PruebaModelo.index]
PruebaModelo['validacionSVC']  = [1 if PruebaModelo.Relevancia[i] == PruebaModelo.prediccionSVC[i] else 0 for i in PruebaModelo.index]

In [ ]:
print('El modelo Naive Bayes obtuvo un',round((PruebaModelo.validacionNB.sum()/len(PruebaModelo.validacionNB))*100,2),"% de precisión")
print('El modelo SVC obtuvo un',round((PruebaModelo.validacionSVC.sum()/len(PruebaModelo.validacionNB))*100,2),"% de precisión")

#### Nube de palabras

In [ ]:
casos_irrelevantes = clasificacion_final[clasificacion_final['relevancia'] == 0]
casos_relevantes = clasificacion_final[clasificacion_final['relevancia'] == 1]

In [ ]:
frases_irrelevantes=list(casos_irrelevantes['frases'])
frases_irrelevantes=" ".join(frases_irrelevantes)
grafico_top(frases_irrelevantes, 2)

In [ ]:
frases_relevantes=list(casos_relevantes['frases'])
frases_relevantes=" ".join(frases_relevantes)
grafico_top(frases_relevantes, 2)

In [ ]:
entrevistas = pd.read_excel('Corpus2.xlsx')

In [ ]:
entrevista_prueba = [token.text.replace('\n', ' ').lower() for token in nlp(entrevistas['Motivo de consulta'][1]).sents]
prueba_prediccion = pd.DataFrame({'oraciones':entrevista_prueba})

In [ ]:
prueba_prediccion.to_excel('resultados/prueba_prediccion.xlsx', index=False)

In [ ]:
vector_prueba = vect.transform(entrevista_prueba)

In [ ]:
prediccion_vector=nb.predict(vector_prueba)

In [ ]:
resultado=pd.DataFrame({'frases':entrevista_prueba, 'relevancia':prediccion_vector})

In [ ]:
resultado.groupby('relevancia').size()